# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [1]:
# Write your code below.
%load_ext dotenv
%dotenv 



In [2]:
import dask.dataframe as dd

c:\Users\suchi\Mini\envs\dsi_participant\lib\site-packages\dask\dataframe\_pyarrow_compat.py:15: FutureWarning: Minimal version of pyarrow will soon be increased to 14.0.1. You are using 12.0.0. Please consider upgrading.
  warnings.warn(


+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [3]:
import os
from glob import glob

# Write your code below.
parquet_files = glob(os.path.join(os.getenv('PRICE_DATA'), "**/*.parquet"), recursive=True)
parquet_files

['../../05_src/data/prices\\ACN\\ACN_2001\\part.0.parquet',
 '../../05_src/data/prices\\ACN\\ACN_2001\\part.1.parquet',
 '../../05_src/data/prices\\ACN\\ACN_2002\\part.0.parquet',
 '../../05_src/data/prices\\ACN\\ACN_2002\\part.1.parquet',
 '../../05_src/data/prices\\ACN\\ACN_2003\\part.0.parquet',
 '../../05_src/data/prices\\ACN\\ACN_2003\\part.1.parquet',
 '../../05_src/data/prices\\ACN\\ACN_2004\\part.0.parquet',
 '../../05_src/data/prices\\ACN\\ACN_2004\\part.1.parquet',
 '../../05_src/data/prices\\ACN\\ACN_2005\\part.0.parquet',
 '../../05_src/data/prices\\ACN\\ACN_2005\\part.1.parquet',
 '../../05_src/data/prices\\ACN\\ACN_2006\\part.0.parquet',
 '../../05_src/data/prices\\ACN\\ACN_2006\\part.1.parquet',
 '../../05_src/data/prices\\ACN\\ACN_2007\\part.0.parquet',
 '../../05_src/data/prices\\ACN\\ACN_2007\\part.1.parquet',
 '../../05_src/data/prices\\ACN\\ACN_2008\\part.0.parquet',
 '../../05_src/data/prices\\ACN\\ACN_2008\\part.1.parquet',
 '../../05_src/data/prices\\ACN\\ACN_200

For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Close:
    
    - `returns`: (Close / Close_lag_1) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [4]:
# Write your code below.
df=dd.read_parquet(parquet_files).set_index("ticker")
dd_feat = df.groupby('ticker',group_keys=False).apply(
    lambda x:x.assign(Close_lag_1=x['Close'].shift(1),
        Adj_Close_lag_1=x['Adj Close'].shift(1),
        Hi_lo_Range=x['High'] - x['Low'],
        Returns=x['Close'] / x['Close'].shift(1) - 1))
dd_feat

C:\Users\suchi\AppData\Local\Temp\ipykernel_39376\260828257.py:3: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  dd_feat = df.groupby('ticker',group_keys=False).apply(


,Date,Open,High,Low,Close,Adj Close,Volume,source,Year,Close_lag_1,Adj_Close_lag_1,Hi_lo_Range,Returns
npartitions=89,,,,,,,,,,,,,
ACN,datetime64[ns],float64,float64,float64,float64,float64,float64,string,int32,float64,float64,float64,float64
AFMD,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZIXI,...,...,...,...,...,...,...,...,...,...,...,...,...
ZIXI,...,...,...,...,...,...,...,...,...,...,...,...,...


+ Convert the Dask data frame to a pandas data frame. 
+ Add a new feature containing the moving average of `returns` using a window of 10 days. There are several ways to solve this task, a simple one uses `.rolling(10).mean()`.

(3 pt)

In [5]:
# Write your code below.
#converting the dask data frame to pandas data frame
pandas_df = dd_feat.compute()
pandas_df

,Date,Open,High,Low,Close,Adj Close,Volume,source,Year,Close_lag_1,Adj_Close_lag_1,Hi_lo_Range,Returns
ticker,,,,,,,,,,,,,
ACN,2017-01-03,117.379997,117.809998,115.820000,116.459999,110.311638,2351600.0,ACN.csv,2017,NaN,NaN,1.989998,NaN
ACN,2017-01-04,116.910004,117.800003,116.430000,116.739998,110.576851,2639800.0,ACN.csv,2017,116.459999,110.311638,1.370003,0.002404
ACN,2017-01-05,116.980003,117.139999,114.949997,114.989998,108.919258,3685400.0,ACN.csv,2017,116.739998,110.576851,2.190002,-0.014991
ACN,2017-01-06,114.989998,116.739998,114.339996,116.300003,110.160088,4125300.0,ACN.csv,2017,114.989998,108.919258,2.400002,0.011392
ACN,2017-01-09,116.139999,116.349998,114.919998,115.000000,108.928719,2550800.0,ACN.csv,2017,116.300003,110.160088,1.430000,-0.011178
...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZIXI,2020-03-26,4.060000,4.530000,3.880000,4.510000,4.510000,1668500.0,ZIXI.csv,2020,4.000000,4.000000,0.650000,0.127500
ZIXI,2020-03-27,4.490000,4.710000,4.100000,4.600000,4.600000,1146800.0,ZIXI.csv,2020,4.510000,4.510000,0.610000,0.019956
ZIXI,2020-03-30,4.830000,4.870000,4.440000,4.640000,4.640000,1212000.0,ZIXI.csv,2020,4.600000,4.600000,0.430000,0.008696


In [10]:
pandas_df.set_index("Date")
# Calculate 10-day rolling average
pandas_df["Rolling_Avg"]= pandas_df['Returns'].rolling(window=10).mean()
# Print the result
print(pandas_df)
#pandas_df.to_csv("C:/Users/suchi/production/OP1.csv")



             Date        Open        High         Low       Close   Adj Close   
ticker                                                                          
ACN    2017-01-03  117.379997  117.809998  115.820000  116.459999  110.311638  \
ACN    2017-01-04  116.910004  117.800003  116.430000  116.739998  110.576851   
ACN    2017-01-05  116.980003  117.139999  114.949997  114.989998  108.919258   
ACN    2017-01-06  114.989998  116.739998  114.339996  116.300003  110.160088   
ACN    2017-01-09  116.139999  116.349998  114.919998  115.000000  108.928719   
...           ...         ...         ...         ...         ...         ...   
ZIXI   2020-03-26    4.060000    4.530000    3.880000    4.510000    4.510000   
ZIXI   2020-03-27    4.490000    4.710000    4.100000    4.600000    4.600000   
ZIXI   2020-03-30    4.830000    4.870000    4.440000    4.640000    4.640000   
ZIXI   2020-03-31    4.600000    4.690000    4.100000    4.310000    4.310000   
ZIXI   2020-04-01    4.11000

Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return? It is not necessary, easy to compute in pandas for data that fits in-memory.
+ Would it have been better to do it in Dask? Why? It would be efficient to do it in Dask when the dataset is very large as it handles large data efficiently via parallelism.

(1 pt)

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.